# Part X — Advanced Topics  
## 42. Advanced Authorization (Policy‑Based Access) — Scalable Permissions That Don’t Rot

“Authentication” tells you *who* the user is. “Authorization” tells you *what they
are allowed to do*.

Most production Django security bugs are **authorization bugs**:
- missing checks
- inconsistent checks (view checks differ from API checks)
- UI hides buttons but backend allows the action
- scoping mistakes (tenant/org boundaries)

This chapter gives you an **industry-grade authorization design** that scales as
your app grows:

- clear roles and permissions vocabulary
- centralized policy functions (PBAC: policy-based access control)
- optional ABAC rules (attribute-based conditions) without chaos
- consistent deny reasons and auditing
- integration across:
  - HTML views
  - DRF APIs
  - templates
  - admin
  - background tasks
- tests that prevent regressions

We’ll apply these patterns to your existing apps:
- `orgs` (Membership roles)
- `tasks` (edit/delete/export rules)
- `articles` (publish rules)

---

## 42.0 Learning Outcomes

By the end you should be able to:

1. Explain RBAC vs ABAC vs PBAC (and choose appropriately).
2. Write a centralized policy layer that answers:
   - “Can actor perform action X on resource Y in tenant Z?”
3. Keep permissions consistent across:
   - HTML views
   - DRF endpoints
   - admin actions
   - background jobs
4. Enforce object-level rules safely (not just model-level perms).
5. Return correct HTTP responses (401/403/404 policy) and log/audit denials.
6. Add custom permissions to models and groups cleanly.
7. Write table-driven permission tests that catch “small refactor broke auth.”

---

# 42.1 Authorization Models (Know the Vocabulary)

## 42.1.1 RBAC (Role-Based Access Control)
- Users have roles (Admin, Member, Editor).
- Roles grant permissions (can_export, can_delete).

**Pros**
- easy to reason about
- admin UI maps well (groups)

**Cons**
- can become rigid (real rules often depend on object attributes)

Your `Membership.Role.ADMIN/MEMBER` is RBAC.

## 42.1.2 ABAC (Attribute-Based Access Control)
Decisions based on attributes like:
- actor: `is_staff`, `department`, `plan_tier`
- object: `status`, `owner_id`, `org_id`
- context: time, IP range, feature flags

Example:
- “Members can edit tasks only if assigned_to == actor OR created_by == actor.”

That’s ABAC.

**Pros**
- expressive, matches real-world rules

**Cons**
- can become messy if rules are scattered

## 42.1.3 PBAC (Policy-Based Access Control) — What we’ll implement
PBAC means:
- You centralize authorization rules in *policy functions/classes*.
- Policies can use roles (RBAC) and attributes (ABAC), but rules live in one place.

PBAC is not a different math model; it’s a **code organization strategy**.

**Industry reason PBAC wins**
- rules don’t drift between UI/API/admin
- tests can target a single policy layer
- you get consistent “deny reasons” and auditing

---

# 42.2 The Authorization Design Goals (What “Good” Looks Like)

A production-ready authorization design should:

1. **Be centralized**: One place to change rules.
2. **Be explicit**: A developer can answer “who can do what” by reading a policy
   module, not hunting across views/templates/serializers.
3. **Be consistent**: Same rule used in HTML view, API endpoint, admin action.
4. **Be testable**: Easy to write tests for each action and edge case.
5. **Support auditing**: Log/record significant permission denials and approvals.

---

# 42.3 Build a Permission Matrix (Do This Before Writing Code)

Write a permission matrix for each domain area. This prevents accidental rule drift.

## 42.3.1 Tasks (Organization-scoped)
Actor: org member with role ADMIN or MEMBER.

Actions:

- `task.view`:
  - ADMIN: allowed
  - MEMBER: allowed
- `task.create`:
  - ADMIN: allowed
  - MEMBER: allowed
- `task.edit`:
  - ADMIN: allowed
  - MEMBER: allowed only if `created_by == actor` OR `assigned_to == actor`
- `task.delete`:
  - ADMIN: allowed
  - MEMBER: denied
- `task.export`:
  - ADMIN: allowed
  - MEMBER: denied

## 42.3.2 Articles (Global content)
- `article.view_published`: anyone
- `article.view_draft`: author or staff
- `article.create`: authenticated
- `article.edit`: author or staff
- `article.publish`:
  - staff only (your chosen policy)

This matrix becomes your “spec.” The policy layer implements it.

---

# 42.4 Implement a Policy Layer (Core Pattern)

We’ll build a small policy framework that returns:
- allow/deny
- a deny reason (for logs/audit and optional error messages)

## 42.4.1 Create `core/policy.py`

```python
from __future__ import annotations

from dataclasses import dataclass
from enum import Enum
from typing import Optional


class Decision(Enum):
    ALLOW = "allow"
    DENY = "deny"


@dataclass(frozen=True)
class PolicyDecision:
    decision: Decision
    reason: str = "ok"

    @property
    def allowed(self) -> bool:
        return self.decision == Decision.ALLOW

    @staticmethod
    def allow(reason: str = "ok") -> "PolicyDecision":
        return PolicyDecision(Decision.ALLOW, reason)

    @staticmethod
    def deny(reason: str) -> "PolicyDecision":
        return PolicyDecision(Decision.DENY, reason)
```

### Why return a structured decision (not just True/False)?
Because “deny reason” is extremely useful for:
- debugging
- user support (“why was I blocked?”)
- audits and security logging
- consistent API error responses

---

# 42.5 Task Authorization Policy (RBAC + ABAC Combined)

We will implement task policies in `tasks/policies.py`.

## 42.5.1 Create `tasks/policies.py`

```python
from __future__ import annotations

from core.policy import PolicyDecision
from orgs.models import Membership
from tasks.models import Task


def can_view_task(*, membership: Membership, task: Task) -> PolicyDecision:
    # Membership implies same org; scoping must still be enforced elsewhere.
    return PolicyDecision.allow()


def can_create_task(*, membership: Membership) -> PolicyDecision:
    return PolicyDecision.allow()


def can_edit_task(*, membership: Membership, actor, task: Task) -> PolicyDecision:
    if membership.role == Membership.Role.ADMIN:
        return PolicyDecision.allow("org_admin")

    if task.created_by_id == actor.id:
        return PolicyDecision.allow("creator")

    if task.assigned_to_id == actor.id:
        return PolicyDecision.allow("assignee")

    return PolicyDecision.deny("not_creator_or_assignee")


def can_delete_task(*, membership: Membership) -> PolicyDecision:
    if membership.role == Membership.Role.ADMIN:
        return PolicyDecision.allow("org_admin")
    return PolicyDecision.deny("not_org_admin")


def can_export_tasks(*, membership: Membership) -> PolicyDecision:
    if membership.role == Membership.Role.ADMIN:
        return PolicyDecision.allow("org_admin")
    return PolicyDecision.deny("not_org_admin")
```

### Key rule: tenant scoping still must be enforced by query
This policy assumes:
- the task already belongs to the org the membership belongs to

**Never** rely on policy alone. Always do:

```python
task = get_object_or_404(Task, id=task_id, organization=org)
```

Because scoping in the query is a second layer of defense.

---

# 42.6 Integrate Policy Into HTML Views (Tasks)

Replace ad-hoc `PermissionDenied` logic with policy calls.

## 42.6.1 Example: task_edit view

```python
from django.core.exceptions import PermissionDenied

from orgs.services import get_membership, get_org_for_user_or_404
from tasks.models import Task
from tasks.policies import can_edit_task


def task_edit(request, org_slug: str, task_id: int):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    task = get_object_or_404(Task, id=task_id, organization=org)

    decision = can_edit_task(membership=membership, actor=request.user, task=task)
    if not decision.allowed:
        raise PermissionDenied

    ...
```

### Why you still raise PermissionDenied (403)
- your policy returns deny reason; you can log it
- HTTP response stays correct and consistent

Optionally log deny reason:

```python
import logging
logger = logging.getLogger(__name__)

if not decision.allowed:
    logger.warning(
        "permission_denied action=task.edit reason=%s actor_id=%s org=%s task_id=%s",
        decision.reason,
        request.user.id,
        org.slug,
        task.id,
    )
    raise PermissionDenied
```

This is extremely useful in production.

---

# 42.7 Integrate Policy Into DRF (API Permissions)

Your DRF permission classes should call the same policy functions.

## 42.7.1 `tasks/api/permissions.py` using policy decisions

```python
from rest_framework.permissions import BasePermission, SAFE_METHODS

from orgs.services import get_membership, get_org_for_user_or_404
from tasks.models import Task
from tasks.policies import can_delete_task, can_edit_task


class TaskObjectPolicyPermission(BasePermission):
    """
    Uses centralized policy functions so HTML and API match.
    """

    def has_object_permission(self, request, view, obj: Task) -> bool:
        if request.method in SAFE_METHODS:
            return True

        org = get_org_for_user_or_404(
            user=request.user,
            org_slug=view.kwargs["org_slug"],
        )
        membership = get_membership(user=request.user, organization=org)

        if request.method in {"PUT", "PATCH"}:
            decision = can_edit_task(
                membership=membership,
                actor=request.user,
                task=obj,
            )
            return decision.allowed

        if request.method == "DELETE":
            decision = can_delete_task(membership=membership)
            return decision.allowed

        return False
```

### Why this is industry standard
- You don’t duplicate “who can edit/delete” rules.
- Frontend, API, background tasks share the same authorization logic.

---

# 42.8 Articles Policy (Draft visibility + publish rules)

## 42.8.1 Create `articles/policies.py`

```python
from __future__ import annotations

from core.policy import PolicyDecision
from articles.models import Article


def can_view_article(*, actor, article: Article) -> PolicyDecision:
    if article.status == Article.Status.PUBLISHED:
        return PolicyDecision.allow("published")

    if not actor.is_authenticated:
        return PolicyDecision.deny("anonymous_cannot_view_draft")

    if actor.is_staff:
        return PolicyDecision.allow("staff")

    if article.author_id == actor.id:
        return PolicyDecision.allow("author")

    return PolicyDecision.deny("not_author_or_staff")


def can_edit_article(*, actor, article: Article) -> PolicyDecision:
    if not actor.is_authenticated:
        return PolicyDecision.deny("not_authenticated")

    if actor.is_staff:
        return PolicyDecision.allow("staff")

    if article.author_id == actor.id:
        return PolicyDecision.allow("author")

    return PolicyDecision.deny("not_author_or_staff")


def can_publish_article(*, actor, article: Article) -> PolicyDecision:
    if not actor.is_authenticated:
        return PolicyDecision.deny("not_authenticated")
    if actor.is_staff:
        return PolicyDecision.allow("staff")
    return PolicyDecision.deny("not_staff")
```

## 42.8.2 Use it in views
Example in article detail:

```python
from django.http import Http404
from articles.policies import can_view_article

def article_detail(request, slug: str):
    article = get_object_or_404(Article, slug=slug)

    decision = can_view_article(actor=request.user, article=article)
    if not decision.allowed:
        # Policy choice: hide drafts from outsiders
        raise Http404

    ...
```

### 404 vs 403 policy (important)
- 404 hides existence (common for drafts and multi-tenant boundaries)
- 403 reveals existence but denies access

Choose per threat model and be consistent. Many content apps use 404 for drafts.

---

# 42.9 Model Permissions and Groups (Use Them, But Know Their Limits)

Django model permissions are coarse-grained and global:
- `articles.change_article` means “can change any article” (global), not “my own”.

Object-level permissions are not built-in (without third-party libs). That’s why you
use policy functions.

## 42.9.1 Add custom permissions in a model
Example: add `can_export_task` permission on Task model.

In `tasks/models.py`:

```python
class Task(models.Model):
    ...
    class Meta:
        permissions = [
            ("export_task", "Can export tasks"),
        ]
```

Run migrations:

```bash
python manage.py makemigrations
python manage.py migrate
```

Now you can:
- assign permission to groups in admin
- check in code:

```python
request.user.has_perm("tasks.export_task")
```

## 42.9.2 When to use model permissions
Use them for:
- staff roles (“Editors can publish”)
- admin-facing capabilities
- coarse global capabilities

Do not use them as your only enforcement for multi-tenant or ownership rules.

**Professional approach:**
- Model perms for “role capability”
- Policy checks for object and tenant-specific rules

---

# 42.10 Policy-Based Access in Templates (UI Hints Only)

Templates should not enforce security (backend must enforce). But templates can use
policies to decide which buttons to show.

## 42.10.1 Pass a boolean to template
In task detail view:

```python
decision = can_edit_task(membership=membership, actor=request.user, task=task)

return render(
    request,
    "tasks/detail.html",
    {
        "task": task,
        "can_edit": decision.allowed,
    },
)
```

Template:

```django
{% if can_edit %}
  <a href="{% url 'tasks:edit' org_slug=organization.slug task_id=task.id %}">Edit</a>
{% endif %}
```

### Why you should not call policy functions directly in templates
You *can* expose them as template tags, but it:
- adds Python logic to presentation layer
- is harder to test
- can hide complexity

Passing booleans from view is usually cleaner.

---

# 42.11 Auditing Authorization (Denied Actions and Sensitive Actions)

For security and compliance, you often want:
- audit log when someone attempts forbidden actions
- audit log when someone performs sensitive actions (exports, role changes)

You already built an `AuditLog` model (optional). Use it here.

## 42.11.1 Audit a denied export attempt
In export start view:

```python
from audit.services import log_event
from tasks.policies import can_export_tasks

decision = can_export_tasks(membership=membership)
if not decision.allowed:
    log_event(
        actor=request.user,
        action="tasks.export.denied",
        target=org,  # or a special audit target model
        details={"reason": decision.reason},
    )
    raise PermissionDenied
```

### Don’t over-audit
Audit:
- exports
- role changes
- permission denials (optional)
- login failures (often via security tooling)

Avoid logging every normal view action; you’ll drown in noise.

---

# 42.12 Testing Authorization Correctly (Table-Driven, High Signal)

The most valuable tests are:
- “outsider cannot access other tenant”
- “member cannot perform forbidden action”
- “admin can perform action”
- “owner can edit; non-owner cannot”

## 42.12.1 Unit tests for policy functions (fast)
Create `tasks/tests_policies.py`:

```python
from django.test import TestCase

from orgs.models import Membership
from tasks.policies import can_edit_task, can_delete_task
from tasks.models import Task
from tests.factories import OrganizationFactory, TaskFactory, UserFactory, MembershipFactory


class TaskPolicyTests(TestCase):
    def test_admin_can_edit_any(self):
        org = OrganizationFactory()
        admin = UserFactory()
        membership = MembershipFactory(
            organization=org,
            user=admin,
            role=Membership.Role.ADMIN,
        )
        task = TaskFactory(organization=org)

        decision = can_edit_task(membership=membership, actor=admin, task=task)
        self.assertTrue(decision.allowed)

    def test_member_can_edit_if_assignee(self):
        org = OrganizationFactory()
        member = UserFactory()
        membership = MembershipFactory(
            organization=org,
            user=member,
            role=Membership.Role.MEMBER,
        )
        task = TaskFactory(organization=org, assigned_to=member)

        decision = can_edit_task(membership=membership, actor=member, task=task)
        self.assertTrue(decision.allowed)
        self.assertEqual(decision.reason, "assignee")

    def test_member_cannot_delete(self):
        org = OrganizationFactory()
        member = UserFactory()
        membership = MembershipFactory(
            organization=org,
            user=member,
            role=Membership.Role.MEMBER,
        )

        decision = can_delete_task(membership=membership)
        self.assertFalse(decision.allowed)
```

These are fast and isolate rule logic.

## 42.12.2 Integration tests (views) for “no bypass”
Still test endpoints because wiring errors can bypass policies.

Example: task delete returns 403 for member.

---

# 42.13 Optional: Object Permissions via django-guardian (When You Need It)

Some teams use `django-guardian` to store object-level permissions in the DB:
- `user has perm change_task on task 123`

**When it’s useful**
- you need “share this specific object with this specific user”
- ACL-style systems (documents shared with specific users)
- lots of per-object permission records

**When it’s not worth it**
- simple ownership/assignee rules
- org role rules
- ABAC logic based on attributes (guardian doesn’t replace those well)

Your current apps (org membership + ownership) are better served by policy
functions—simpler and often faster.

If you want, I can add a full guardian-based section later, but PBAC + ABAC via
policies is already an industry-standard approach.

---

# 42.14 Exercises (Do These Before Proceeding)

1. **Unify rules across HTML and API**
   - Ensure your Task API delete/edit uses the same `tasks.policies` as HTML views.
   - Write a test proving both return forbidden for the same user.

2. **Add a new role**
   - Add Membership role `MANAGER`
   - New rule: MANAGER can export but cannot delete
   - Update policy functions and add tests

3. **Add a context-based ABAC rule**
   - Example: Members cannot edit tasks after they are DONE unless admin
   - Update `can_edit_task` to deny when `task.status == DONE` for members
   - Add tests

4. **Audit sensitive actions**
   - Add audit logs for:
     - tasks export started
     - tasks export downloaded
     - task deleted
   - Ensure logs include actor_id, org_id, request_id

5. **Document your permission matrix**
   - Create `docs/authorization.md` listing actions and rules
   - Include which policy functions enforce them

---

## 42.15 Chapter Summary

- Authorization must be centralized to prevent drift and security bugs.
- PBAC (policy layer) is an implementation strategy that can combine RBAC + ABAC.
- Always scope queries by tenant/org and then apply object-level policies.
- Reuse the same policy functions across HTML views, DRF APIs, admin actions, and tasks.
- Return consistent HTTP semantics (401/403/404) according to your chosen policy.
- Test authorization rules directly (unit tests) and via endpoints (integration tests).
- Audit sensitive actions and (optionally) permission denials for operational clarity.

---

Next chapter: **43. Internationalization (i18n) and Localization (l10n)**  
We’ll add multilingual support, translate templates, handle time zones correctly,
and implement translation workflows that match real production teams.